PEST++ V3            |  Applied GW  |  Calibration and UA
:-------------------------:|:-------------------------:|:-------------------------:
[<img src="imgs/pest++v3_cover.jpg" style="float: left" width=300>](https://pubs.usgs.gov/tm/07/c12/tm7c12.pdf)  |  [<img src="imgs/appliedGW.jpg" style="float: center" width=300>](https://www.elsevier.com/books/applied-groundwater-modeling/anderson/978-0-08-091638-5)  |  [<img src="imgs/cal_ua_cover.jpg" style="float: right" width=300>](https://pesthomepage.org/pest-book)
</br>



# 1. Introduction to PEST (Parameter ESTimation Tool)

## 1.1 PEST Algorithm
The search algorithm used in PEST is the ___Gauss-Levenberg-Marquardt algorithm (GLMA)___. The central feature of the PEST engine is the GLMA search algorithm that iteratively optimizes the model parameters to improve its fit to observed data and other objectives. The fit to the observations is hereby expressed through the Measurement Objective Function. In the simplest case, this will be the weighted sum of squares of the residuals between measurement and simulation results:

$$\Phi = {\sum_iw_i(h_i^{obs}-h_i^{sim})^2}$$
where (h<sup>obs</sup> denotes an observation (typically from a field measurement), h<sup>sim</sup> its related simulation result, and _w_ the weight that has been applied to the measurement.


### 1.1.1 What are the driving forces behind the algorithm?

The GLMA changes the model parameters until a minimum objective function value is found. Running PEST, the user will observe two working steps per iteration:

- Derivative calculation: The parameters are changed incrementally. By repeating the model run for each parameter, and observing the resulting changes of observation values, the partial derivative for each pair of parameter and observation can be calculated by finite-difference approximation. These derivatives form the elements of the ___`Jacobian matrix`___. The numerical effort to calculate the Jacobian matrix usually dominates the iteration. PEST has the option for switching from a 2, 3 and 5 point derivative calculation.
<img src="imgs/jmatrix.PNG" style="inline" width=600>

$$
J_{ij} = \frac{\partial o_i}{\partial p_j} = 
 \begin{bmatrix}
 \partial o_1/\partial p_1 & \partial o_1/\partial p_2 & \cdots & \partial o_1/\partial p_j \\
 \partial o_2/\partial p_1 & \partial o_2/\partial p_2 & \cdots & \partial o_2/\partial p_j \\
  \vdots  & \vdots  & \ddots & \vdots  \\
 \partial o_i/\partial p_1 & \partial o_i/\partial p_2 & \cdots & \partial o_i/\partial p_j \\
 \end{bmatrix}
$$





- The parameter values are adjusted aiming to reduce the objective function. The direction and magnitude of the adjustment is expressed by the ___`parameter upgrade vector`___. To identify the optimal direction of this vector, the GLMA uses a combination of two strategies:
    - While the objective function shows a predominantly linear behaviour, the method of `gradient descent` is applied. This method determines the parameter upgrade vector from the direction of steepest descent of the objective function. This can often be observed during the initial phase of the optimization.
    - Objective-function nonlinearity is addressed via the `Gauss-Newton method`. This method computes a parameter upgrade vector based on the presumption of a quadratic behaviour of the objective function.

The two methods are not mutually exclusive: The GLMA interpolates between them, controlled by a scaling parameter named as the Marquardt-Lambda.

### 1.1.2 High Lambda vs. Small Lambda

PEST dynamically updates lambda depending on the progress in reducing the objective function. The current lambda during the PEST run is a good indicator for the current nonlinearity of the objective function.
- high lambda values (e.g., > 10) indicate linear behaviour (and predominant use of the gradient descent method).
- small lambda values (e.g., < 2) indicate nonlinear behaviour (and predominant use of the Gauss-Newton method).

Figures below illustrate the development of the objective function and the Marquardt-Lambda during a typical PEST optimization. Gradient descent is used in the first iterations, indicated by higher lambda values. When the objective function approaches its (local) minimum, Lambda falls to near zero indicating almost exclusive use of the Gauss-Newton method.
<img src="imgs/fig.01_PhiLambdaCompare.png" width=500 style="float: center">
<center>Fig. Development of the objective function and the Marquardt lambda during a PEST run.</center>
<center>source: FePEST 7.1 Documentation</center>
<img src="imgs/fig.02_PhiLambdaMap.png" width=500 style="float: center">
<center>Fig. Schematic illustration of contours of the objective function and the path of the parameter upgrades vectors (after Doherty).</center>



## Are you satisified already with a PEST result?

If successful, the GLMA will find a parameter set that constitutes a local minimum of the defined objective function. This is an important restriction because multiple local minima might be present, and ___`it is not guaranteed that the one found is also the global minimum.`___

It is therefore possible that different PEST runs result in different parameter sets if the iteration starts at different initial parameter values. These should therefore be chosen as close as possible to those values that are expected.

<mark>___The modeller should also critically review the resulting parameter set and the model-to-measurement-misfit. Strong, but also very low departures indicate potential problems with the optimization.___<mark/>

# 2 Setting up PEST:  Communicating with the model 
*Source: PEST workshop in 2018*<br/>

## 2.1 Manual Trial-and-Error

Recall that, for all its sophistication, parameter estimation calibration does the same things as a modeler does in manual trial-and-error.  Here is a flow chart manual trial-and-error history matching from *Applied Groundwater Modeling (2nd edition)* by Anderson et al. (2015): 


<img src="imgs/Fig9.1_manual_t&e.png" style="float: center">

## 2.2 Automatic Parameter Estimation

### Compare this to the full overdetermined parameter estimation flow chart:


<img src="imgs/Fig9.9_full_overdetermined_PE_flowchart.png" style="float: center">

To be more specific, in the blue box in Figure 9.9 above there are certain steps that occur before and after each forward model run that PEST does.  They exactly equate to what a manual trial-and-error calibration does, but PEST does it for you! Here are the steps that happen:

<img src="imgs/Fig9.8_PE_flowchart.png" style="float: center">

# 2. Middle Bosque SWAT-MODFLOW Model
## MODFLOW
- It is a heterogenous 3D areal (two layers) model that is used for its SWAT-MODFLOW model.
- 5 zones for 5 Hydraulic conductivities and 5 Specific Yields
- Grid size is 300 by 300 meters
- 268 columns and 154 rows

<img src="imgs/fig.00_mb_modelmuse.JPG">

## SWAT    
- 69 subbasins
- 1,693 HRUs

## SWAT-MODFLOW model
- To link the SWAT with MODFLOW models, we used QSWATMOD (https://github.com/spark-brc/QSWATMOD2)




# 3. Parameterization

## 3.1 Spatial Parameterization (Zonal Approach & Pilot Points)
According to characteristics of a model domain, either zones of piecewise constancy (zonal approach), pilot points, or a combination of these can be used in a spatial parameterization [[Dotherty and Hunt, 2010_PEST_Groundwater]].
- Based initial results (baseflow, peakflow, watertable elevation and their timings)and the locations of observation data, several parameters can be used for optimizing the model.
![title](imgs/fig.01_obs.JPG)
___Fig. 01 Locations of measurements___
</br>
</br>

![Fig. 02 River Parameters](imgs/fig.02_riv_par.JPG)
___Fig. 02 River Parameters___


- 8 different types of parameters for 4 different zones from SWAT (Total: 40)
- 5 Hydraulic Conductivity / 5 Specific Yield Parameters from 5 zones in MODFLOW (Total: 10)
- 4 Riverbed conductance / 4 Riverbed bottom elevation parameters from 4 combined subs (Total: 8)
> Total 64
[Fig. Parameterize](Images/okvg_par.jpg)

![title](imgs/zon_pp.jpg)





## 3.2 Workflow

<img src="imgs/fig.03_workflow.JPG" width=500 style="float: left;margin:0px 30px 0px 0px">

___Fig. 02 River Parameters___


Adapted from [[White et al., 2016_pyemu]]; [[Park, 2018]]; [[Liu et al., 2019]];  [[Park et al., 2020_PEST]]
- Create template files
	Template files are ...
	- Read *.pval file from MODFLOW inputs and create *.tpl file for it
	- Read the "model.in" file from SWAT inputs and create the "model.in.tpl" file (See Appendix)
	- Parameterize riverbed conductance and river bottom elevation
		- Include a column for Subbasin numbers or sub group () in an existing *.riv file (River Package)
			[Fig. eg_riverpackage](Images/eg_river_package.png)
		-  The following code demonstrates how to create the "mf_riv.par" file

We want to run PEST so it does what you were doing by hand.  To do this we need to provide conduits that change a model input file and that extract model results after the forward run finishes. First we'll do some Python notebook prep (push shift-enter in the next code block)

# 4. What are the *.template(.tpl) and instruction(.ins) files?

## 4.1 Template files are used to create model input
<img src="imgs/fig.04_eg_tpl.JPG" style="float: center">

<mark>Template files simply replace parameter numerical values with a code variable, named in the PEST Control File.</mark>
- There needs to be one template file __for each model__ input file that has parameters that we want PEST to estimate.
- PEST will read in each template file,  use it as a template to substitute its updated parameter value, then write one model input file for each TPL file it read.
- In the PEST control file we specify each template file __and__ the associated model model input file we want PEST to create after it has updated estimates for the parameters on separate lines.
- So, say we had a MODFLOW input file named 'mf_300.pval' for which we made a template file 'mf_300.pval.tpl'. In the "model input/output" section of the PEST control file there will be a line containing this:

mf_300.pval &nbsp;&nbsp; mf_300.pval.tpl

### 4.1.1 Rules for constructing TPL Files 

 1. The first line of the TPL file must identify that it is a template file by listing "`ptf ~`" where "`~`" is a "parameter delimiter" that tells PEST where a parameter sits in the file. We used a tilde here but it can be any symbol. __However__, whatever delimiter symbol is listed in the first line must be used consistently throughout that template file.
 2. The template file looks exactly like the original model input file __BUT__ parameters are substituted for the  model input(s) that we want PEST to estimate.  Parameters are identified by surrounding the parameter name listed in the PEST control (.pst) file with the delimiter.  For the "`~`" delimiter that we used above, and a horizontal K parameter named "`kc`" listed as a parameter in a PEST .pst file, the template file would have "`~   kc ~`" __wherever that Kh value__ was listed in the original model input file. 
   * Note that the parameter name can be anywhere between the parameter delimiters
   * PEST will fill the space up to and including the parameter delimiters with a value, so make them as wide as possible for maximum precision
   
#### So
"`~    kc    ~`" will be replaced by the value for `kc` specified by PEST. If that value is 3.14, PEST will write "`3.14000000000`" in its place.

#### The PEST manual explains more detail about how you can control the writing of model input (e.g. scientific notation, double precision, etc.); see http://www.pesthomepage.org/Downloads.php to get the most recent version of the manual.

## 4.2 Instruction files extract results from model output

Similar to the template files, the names of instruction files and which model output files they should work on are listed after all the template files in the * model input/output section of the PEST control file.  As you might expect with the wide range of model output file variation, creating instruction files is slightly more complicated than creating template files. There is basically an __internal scripting language__ for reading text files of model output, extracting the output of interest, and providing it directly to PEST.

<img src="imgs/fig.05_eg_ins.JPG" style="float: center">

Similar to the template files, the names of instruction files and which model output files they should work on are listed after all the template files in the * model input/output section of the PEST control file.  As you might expect with the wide range of model output file variation, creating instruction files is slightly more complicated than creating template files. There is basically an internal scripting language for reading text files of model output, extracting the output of interest, and providing it directly to PEST.

### Rules for INS Files 

 * The first line on an .ins file must be "`pif ~`" where "`~`" is a "marker delimiter"--a symbol that can be used to identify text to search for.  It is expected on this first line but it's not always used.
 * The scripting options are extensive but particular. Some options on how to navigate to the numerical data you want to read are:
   1. Using a line advance.  PEST starts at the very top the model output file, and you have to move down 1 line to get to the first line that you see in the model output. You can tell PEST to move down the file _`n`_ lines using the `l` character (=lowercase letter l) with a number.  So "`l1`" moves down one line, "`l3`" moves down 3 lines.  
   2. Using the marker delimiter, the INS file can search through a file until it finds a "primary marker". For example:  
   "`~VOLUMETRIC BUDGET FOR ENTIRE MODEL~`" can be used to search for budgets in a LST file  
   This is particularly well suited for output files (like a LST file) that have unpredictable lengths.  Note though that PEST will always start at the top of the file and go down, never up and never wrapping once it reaches the end.  This can be a problem when the order of some observations with respect to other observations is not consistent (e.g., some MODPATH output).  When searching for mutiple observations that may vary in order in an output file, it is easiest to have multiple instruction files open the same model output file multiple times so you are always starting at the top of the file (PEST does not mind). 
   3. Next, you can search for a "secondary marker" within a line using the marker delimiter again. This navigates from the left of the line until the secondary marker is found.
   4. Once on the line you can specify which columns on a line to extract.  So a line in an instruction file that says '~101  138~ (depth_T2-90)46:58'means that PEST will look for '101  138' in the model output file (with the exact number of spaces between the two numbers) then extract column 46:58 to determine the model output that equates to the target observation 'depth_T2-90' that is listed in the PEST control file.   
5. Finally, you can read in whitespace-delimited numerical data using "`!`" around the observation name:  
   for example, if the output file is:  
   ```
   Output file from run 5
   Run date: 1/1/2012
   Run time: 24.3 hours
   Converged in 350 iterations
   Head Observations:
   H1=33.345 H2=45.34
   ...
   ```  
   The instruction file would be like 
   ```
   pif ~
    ~Head Observations~
    l1 ~H1=~ !h1val! ~H2=~ !h2val!
   ```
   
 These are only a few of the most commonly used options but more options, and more detail on these, are available in the PEST manual.  
   
   Let's check an instruction file using `INSCHEK`, a handy utility that allows us to check our instruction files without having to do a full PEST run. You can see what INSCHEK is looking for by simply typing 'INSCHEK" at the command line.  You'll see: 
   

# 5. How can we create those files?

### Another manual Trial-and-Error
<img src="imgs/seriously.gif" style="float: center">